In [1]:
import chromadb
import webvtt
import requests
import time
from bs4 import BeautifulSoup
import polars as pl
import patito as pt
from pydantic import BaseModel, Field, TypeAdapter, ValidationError
import pytest
import helpers 
from enum import Enum
from typing import Optional
from tqdm import tqdm
from config import *

pl.Config(tbl_cols=-1)
pl.Config(set_tbl_width_chars=300)

In [2]:
df_old = pl.read_csv(r"C:\repos\ATP_AI_Answers\data\atp_web_scraping\transcript.csv")
df_old = helpers.get_pre_scraping_df(df_old)

print(len(df_old))

scraped_records = helpers.get_records_from_html_file(
    r"C:\repos\All_The_Preaching_Web_Scraping_Pipeline\data\atp_archive_2025-07-07.html"
)
df_new = helpers.get_pre_scraping_df(scraped_records)
print(len(df_new))

df_diff = df_old.join(df_new, "video_id", "left", coalesce=False)

df_diff = df_diff.with_columns(
    [
        pl.coalesce(pl.col("video_id_right"), pl.col("video_id")).alias("video_id"),
        pl.coalesce(pl.col("section_right"), pl.col("section")).alias("section"),
        pl.coalesce(pl.col("preacher_right"), pl.col("preacher")).alias("preacher"),
        pl.coalesce(pl.col("title_right"), pl.col("title")).alias("title"),
        pl.coalesce(pl.col("video_url_right"), pl.col("video_url")).alias("video_url"),
    ]
).drop(
    [
        "video_id_right",
        "section_right",
        "preacher_right",
        "title_right",
        "video_url_right",
    ]
)
with pl.Config(tbl_cols=-1) and pl.Config(set_tbl_width_chars=200):
    print(df_diff)

2 validation errors for PreScrapingDataFrameModel
video_id
  28 rows with duplicated values. (type=value_error.rowvalue)
video_url
  28 rows with duplicated values. (type=value_error.rowvalue)
15962
16300
shape: (15_962, 5)
┌──────────┬─────────────────────────────────┬─────────────────────────┬─────────────────────────────────┬─────────────────────────────────┐
│ video_id ┆ section                         ┆ preacher                ┆ title                           ┆ video_url                       │
│ ---      ┆ ---                             ┆ ---                     ┆ ---                             ┆ ---                             │
│ i64      ┆ str                             ┆ str                     ┆ str                             ┆ str                             │
╞══════════╪═════════════════════════════════╪═════════════════════════╪═════════════════════════════════╪═════════════════════════════════╡
│ 3594425  ┆ fbbf 2019                       ┆ pastor steven anderson  

In [3]:
df_diff = df_diff.unique(subset=["preacher"]).sort(by=pl.col("preacher"))

df_diff.write_csv(
    r"C:\repos\All_The_Preaching_Web_Scraping_Pipeline\data\metadata_check.csv"
)

In [4]:
# import json
# from chromadb import QueryResult, PersistentClient, Settings
# import numpy as np
# import os

# # --- ChromaDB Client Setup ---
# # This code runs ONCE when the server starts.
# # Construct the path to the database directory at the project root.
# # This makes the path independent of where the script is run from.

# # project_root = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# # chroma_db_path = os.path.join(project_root, ".chroma")

# chroma_db_path = r"C:\repos\All_The_Preaching_Web_Scraping_Pipeline\.chroma" #temporary while using notebook

# client = PersistentClient(
#     path=chroma_db_path,
#     settings=Settings(
#         is_persistent=True,
#         persist_directory=chroma_db_path,
#         anonymized_telemetry=False,
#     ),
# )

# transcript_collection = client.get_or_create_collection(name="atp_transcripts")
# chunk_collection = client.get_or_create_collection(name="atp_chunks")